<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
!pip uninstall torch

In [ ]:
!pip install torch==2.0.0+cu117 

In [4]:
# !pip install gensim nltk spacy

In [131]:
# https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
from gensim.models.keyedvectors import KeyedVectors
from numpy.linalg import norm
import numpy as np
import re
import nltk
from nltk import WordPunctTokenizer
from nltk import SpaceTokenizer
import pandas as pd
from functools import cmp_to_key
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
import re
import string


In [6]:
# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download('stopwords')

In [7]:
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

In [8]:
wv_embeddings.index_to_key

['use',
 'code',
 'using',
 'like',
 'will',
 'want',
 'need',
 'get',
 'file',
 'one',
 'just',
 'data',
 'way',
 '1',
 'also',
 'function',
 'problem',
 'error',
 '-',
 'example',
 '2',
 'class',
 'work',
 'value',
 'method',
 'first',
 'set',
 'see',
 'following',
 '0',
 'user',
 'make',
 'new',
 'add',
 'know',
 'try',
 'time',
 'something',
 'http',
 'trying',
 'create',
 'object',
 'now',
 'page',
 'server',
 'string',
 'help',
 'app',
 'list',
 'table',
 'application',
 'run',
 'two',
 'call',
 'change',
 'works',
 'used',
 'type',
 '3',
 'array',
 'find',
 'name',
 'html',
 'tried',
 'case',
 'gt',
 'lt',
 'line',
 'instead',
 'different',
 'question',
 'thanks',
 'however',
 'working',
 'java',
 'files',
 'view',
 'solution',
 'php',
 'another',
 'check',
 'project',
 'values',
 'database',
 'think',
 'sure',
 'number',
 'text',
 'image',
 'may',
 'query',
 'possible',
 'without',
 'id',
 'able',
 'since',
 'look',
 'version',
 'variable',
 'found',
 'issue',
 '4',
 'please',


#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [13]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [14]:
cosine_similarity([wv_embeddings['dog']], [wv_embeddings['dog']])

array([[1.0000001]], dtype=float32)

In [15]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


In [16]:
dog_v = wv_embeddings[word]

Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?


#### Ответ 1: Кот не в топ 5. Место 25.

In [18]:
# С помощью встроенной функции
def get_most_similar_words_gensim(embeddings, word, n_words):
    return embeddings.most_similar(positive=[word], topn=n_words)

In [19]:
top5_sim_dog = get_most_similar_words_gensim(wv_embeddings, 'dog', 5)
top100_sim_dog = get_most_similar_words_gensim(wv_embeddings, 'dog', 100)
cat_pos_top5 = [index for (index, x) in enumerate(top5_sim_dog) if x[0] == 'cat']
if cat_pos_top5 : 
    cat_pos_top5[0]
else:
    print('Кот не в топ 5')
    
cat_pos_top100 = [index for (index, x) in enumerate(top100_sim_dog) if x[0] == 'cat']
if cat_pos_top100:
    print(f'Позиция кота {cat_pos_top100[0]}')
    

Кот не в топ 5
Позиция кота 25


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [20]:
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        self.tokenizer = WordPunctTokenizer()
    def tokenize(self, text):
        return self.tokenizer.tokenize(text)


In [93]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    tokens = tokenizer.tokenize(question)
    vectors = [embeddings[x] for x in tokens if x in embeddings]
    
    if len(vectors) == 0 :
        return np.zeros(dim)
    
    stacked_v = np.stack(vectors)

    return np.mean(stacked_v, axis=0)

<!-- Теперь у нас есть метод для создания векторного представления любого предложения. -->

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

#### Ответ 2: -1.29

In [94]:
question = 'I love neural networks'
tokenizer = MyTokenizer()

round(question_to_vec(question, wv_embeddings, tokenizer)[2], 2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### Ответ 3: max(HITS@47 - DCG@1) =max(HITS@47) - min(DCG@1) = 1
* max(DCG@1) = 1
* max(Hits@47) = 1
* min(DCG@1) = 0

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой) (N = 1 ?)

#### Ответ 4: 0.3

In [95]:
round(1 / np.log2(10), 1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [96]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    return sum([1 if x <= k else 0 for x in dup_ranks]) / len(dup_ranks)

In [97]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    return sum([1 / np.log2(x + 1) if x <= k  else 0 for x in dup_ranks]) / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [98]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [99]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

Считайте данные.

In [100]:
def read_corpus(filename, val=False):
    data = []
    for line in open(filename, encoding='utf-8'):
        splitted = line.split('\t')
        data.append((splitted[0], splitted[1:]))
    return data

Нам понадобиться только файл validation.

In [101]:
validation_data = read_corpus('./data/validation.tsv')

In [102]:
train_data = read_corpus('./data/train.tsv')

Кол-во строк

In [103]:
len(validation_data)

3760

Размер нескольких первых строк

In [104]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 2
2 2
3 2
4 2
5 2


In [105]:
q, ex = validation_data[0]
ex

['How do you best convert a recursive function to an iterative one?',
 'How can i use ng-model with directive in angular js',
 'flash: drawing and erasing',
 'toggle react component using hide show classname',
 'Use a usercontrol from another project to current webpage',
 '~ Paths resolved differently after upgrading to ASP.NET 4',
 'Materialize datepicker - Rendering when an icon is clicked',
 'Creating PyPi package - Could not find a version that satisfies the requirement iso8601',
 'How can I analyze a confusion matrix?',
 'How do I declare a C array in Swift?',
 'Using rand() when flipping a coin and rolling a die',
 'Handling a JSON field with a special character in its name in Java',
 'React Native select row on ListView when push it',
 "Get 'creation_time' of video using ffmpeg and regex",
 'Does row exist and multiple where',
 "How to specify a classifier in a gradle dependency's dependency?",
 'Using $unwind on multiple documents',
 'Visual Studio Code: Missing X509Certificate

In [106]:
[x for x in train_data if x[0] == 'Sending array via Ajax fails'] 

[('Sending array via Ajax fails',
  ['Getting all list items of an unordered list in PHP\n'])]

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [107]:
# from copy import deepcopy ?????

In [108]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vec = question_to_vec(question, embeddings, tokenizer, dim)
    candidates_vecs = [(index, question_to_vec(x, embeddings, tokenizer, dim)) for (index, x) in enumerate(candidates)]
    return sorted(enumerate(candidates), key=lambda x: -cosine_similarity([candidates_vecs[x[0]][1]], [question_vec])[0])

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [109]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [110]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]
[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]


Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [111]:
# должно вывести
#  results = [[(1, 'C# create cookie from string and send it'),
#             (0, 'Convert Google results object (pure js) to Python object'),
#             (2, 'How to use jQuery AJAX for an outside domain?')],
#            [(1, 'WPF- How to update the changes in list item of a list'), #скрыт
#             (0, 'Getting all list items of an unordered list in PHP'), #скрыт
#             (2, 'select2 not displaying search results') #скрыт
#             ]] 

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ 5: 102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [112]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [113]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.276 | Hits@   1: 0.276
DCG@   5: 0.332 | Hits@   5: 0.386
DCG@  10: 0.349 | Hits@  10: 0.438
DCG@ 100: 0.395 | Hits@ 100: 0.671
DCG@ 500: 0.421 | Hits@ 500: 0.875
DCG@1000: 0.434 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [114]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [117]:
class NoStopsPuncTokenizer:
    def __init__(self, lang = 'english'):
        self.tokenizer = WordPunctTokenizer()
        self.lang = lang
        self.stops = set(stopwords.words(self.lang))
        
    def tokenize(self, text):
        return [x for x in self.tokenizer.tokenize(text) if x not in self.stops]

stops_filter_tokenizer_punct = NoStopsPuncTokenizer()

In [120]:
class NoStopsPuncStemmaTokenizer:
    def __init__(self, lang = 'english'):
        self.tokenizer = WordPunctTokenizer()
        self.lang = lang
        self.ps = PorterStemmer()
        self.stops = set(stopwords.words(self.lang))
        
    def tokenize(self, text):
        return [self.ps.stem(x) for x in self.tokenizer.tokenize(text) if x not in self.stops]

no_stops_stemma_tonekizer = NoStopsPuncStemmaTokenizer()

In [134]:
class NoStopsNoPuncTokenizer:
    def __init__(self, lang = 'english'):
        self.tokenizer = WordPunctTokenizer()
        self.lang = lang
        self.stops = set(stopwords.words(self.lang))
        
    def tokenize(self, text):
        return [x for x in self.tokenizer.tokenize(text) if x not in self.stops and x not in string.punctuation]

stops_filter_tokenizer_no_punct = NoStopsNoPuncTokenizer()

In [121]:
window = 10 # В окно 10 в большинстве случаев будут попадать слова как из началаьного вопроса, так и из похожего.

In [122]:
sentences = [x[0] + ' ' +  x[1][0] for x in train_data]

In [139]:
words = [no_stops_stemma_tonekizer.tokenize(x) for x in tqdm(sentences)]
words_len = 0
for x in words:
    words_len+= len(x)
print(words_len)

  0%|          | 0/1000000 [00:00<?, ?it/s]

14884932


In [124]:
no_stops_words_punct = [stops_filter_tokenizer_punct.tokenize(x) for x in tqdm(sentences)]
no_stops_words_len = 0
for x in no_stops_words_punct:
    no_stops_words_len+= len(x)
print(no_stops_words_len)

  0%|          | 0/1000000 [00:00<?, ?it/s]

14884932


In [135]:
no_stops_no_punct_words = [stops_filter_tokenizer_no_punct.tokenize(x) for x in tqdm(sentences)]
l2 = 0
for x in no_stops_no_punct_words:
    l2 += len(x)
print(l2)

  0%|          | 0/1000000 [00:00<?, ?it/s]

12675976


In [165]:
embeddings_trained = Word2Vec(no_stops_words_punct, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=15).wv


In [166]:
govno2 = (embeddings_trained['list'] + embeddings_trained['array'])/2
cosine_similarity([govno2], [embeddings_trained['list']]), cosine_similarity([govno2], [embeddings_trained['array']]) 


(array([[0.8433889]], dtype=float32), array([[0.889202]], dtype=float32))

In [170]:
top200 = get_most_similar_words_gensim(embeddings_trained, 'list', 300000)
s_pos_top200 = [index for (index, x) in enumerate(top200) if x[0] == 'array']
if s_pos_top200:
    print(f'Позиция array {s_pos_top200[0]}')

Позиция array 4


In [171]:
get_most_similar_words_gensim(embeddings_trained, 'list', 5)

[('List', 0.6848503351211548),
 ('lists', 0.6496286392211914),
 ('sublist', 0.6007610559463501),
 ('tuple', 0.5218208432197571),
 ('array', 0.5041189193725586)]

In [168]:
print(f"Num of words: {len(embeddings_trained.index_to_key)}")

Num of words: 49363


In [169]:
wv_ranking = []
max_validation_examples = 2000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ex = ex[0]
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.320 | Hits@   1: 0.320
DCG@   5: 0.407 | Hits@   5: 0.484
DCG@  10: 0.436 | Hits@  10: 0.573
DCG@ 100: 0.487 | Hits@ 100: 0.821
DCG@ 500: 0.505 | Hits@ 500: 0.961
DCG@1000: 0.509 | Hits@1000: 1.000


Лучший результат: min_count=5, window=15, WordPunctTokenizer с филдьтрацией стоп слов и стеммингом
Результаты (K, DCG, HITS): [(1, 0.412, 0.412), (5, 0.5, 0.576), (10, 0.523, 0.649), (100, 0.567, 0.859), (500, 0.582, 0.972), (1000, 0.585, 1.0)]

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:

### 1. Токенизация по словам с пунктуацией, с удалением стоп слов. Токенизация больше чем по одну слову на токен не рассматривалась, так как работаем с предложениеми.  С фильтром стоп слов качество лучше, для векторных представлений вопросов они только мешали.

### 2. Да, полмог стемминг слов.

### 3. Обучать с помощью Word2Vec на вопросах из train лучше, потому что можно сильнее учитывать контекст в построении эмбеддингов. 

### 4. Проблема в том, как мы составляем эмбедденги предложений. Среднее по эмбеддингам не лучший способ.

### 5. Лучше составлять эмбедденги предложений. (Взять от более сложной модели)


# Bonus track: Grid Search

In [55]:
def tokenize_sentences(sentences, tokenizer):
    tokenized_sentences = [tokenizer.tokenize(x) for x in tqdm(sentences)]
    l = 0
    for x in tokenized_sentences:
        l += len(x)
    print(f'Всего токенов получилось: {l}')
    return tokenized_sentences

def train_embeddings(tokenized_sentences, min_count, window):
    embeddings_trained = Word2Vec(tokenized_sentences,
                                  vector_size=200,
                                  min_count=min_count,
                                  window=window).wv
    return embeddings_trained


def get_results(validation_data, embeddings_trained, tokenizer, max_val_examples):
    wv_ranking = []
    max_validation_examples = max_val_examples
    for i, line in enumerate(validation_data):
        if i == max_validation_examples:
            break
        q, *ex = line
        ex = ex[0]
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)

    results = []
    for k in [1, 5, 10, 100, 500, 1000]:
        dcg = dcg_score(wv_ranking, k)
        hits = hits_count(wv_ranking, k)
        results.append((k, round(dcg,3), round(hits, 3)))
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg, k, hits))

    return results


def word2VecGridSearch(sentences, val_data,  min_counts, windows, tokenizers, max_val_examples):
    """
    :return: the best combination of parameters with results
    parameters format ([min_count, window, tokenizer])
    results format [[k], [dcg], [hits]]
    """
    results = []
    for tokenizer in tokenizers:
        tokenized_sentences = tokenize_sentences(sentences, tokenizer)
        for min_count in min_counts:
            for window in windows:
                print()
                print(f'mc = {min_count}, w = {window} t = {tokenizer.__class__}')
                print('----------------------------------------------------------')
                embeddings_trained = train_embeddings(tokenized_sentences, min_count, window)
                results.append(((min_count, window, tokenizer.__class__),get_results(val_data, embeddings_trained, tokenizer, max_val_examples)))
                print()

    best = max(results, key=lambda x: x[1][0][1])
    print(f'Лучший: mc = {best[0][0]}, w = {best[0][1]}, tokeizer={best[0][2]}')
    print(f'Результаты: {best[1]}')

    return results

In [ ]:
grid_search_result = word2VecGridSearch(sentences, 
                   validation_data, 
                   [5, 10], 
                   [5, 10, 15], 
                   [stops_filter_tokenizer_punct, no_stops_stemma_tonekizer], 
                   2000)